# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load the dataset
df = pd.read_csv('tic-tac-toe.csv')

In [2]:
# Inspect the dataset
print(df.head())
print(df.info())

  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB
None


In [9]:
# Convert categorical values to numeric
label_encoder = LabelEncoder()
for column in df.columns:
    df[column] = label_encoder.fit_transform(df[column])

In [10]:
# Separate inputs (X) and output (y)
X = df.drop('class', axis=1)
y = df['class']

In [11]:
# Normalize the input data
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

In [12]:
df.shape

(958, 10)

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

In [23]:
# Create the Sequential model
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(2, activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
# Fit the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

Epoch 1/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4702 - loss: 0.7321 - val_accuracy: 0.5365 - val_loss: 0.6779
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 687us/step - accuracy: 0.6611 - loss: 0.6436 - val_accuracy: 0.6406 - val_loss: 0.6368
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 608us/step - accuracy: 0.6476 - loss: 0.6330 - val_accuracy: 0.6719 - val_loss: 0.6174
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 681us/step - accuracy: 0.6509 - loss: 0.6281 - val_accuracy: 0.6771 - val_loss: 0.6027
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 627us/step - accuracy: 0.6769 - loss: 0.5897 - val_accuracy: 0.7031 - val_loss: 0.5909
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 618us/step - accuracy: 0.7207 - loss: 0.5649 - val_accuracy: 0.7083 - val_loss: 0.5781
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 657us/step - accuracy: 0.7288 - loss: 0.5577 - val_accuracy: 0.7135 - val_loss: 0.5699
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 633us/step - accuracy: 0.7352 - loss: 0.5486 - val_accuracy: 0.73

In [25]:
# Save the model in the native Keras format
model.save('tic-tac-toe.keras')

In [27]:
# Load the model
loaded_model = tf.keras.models.load_model('tic-tac-toe.keras')

# Recompile the model (if needed)
loaded_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Evaluate or train the model to use the metrics
loaded_model.evaluate(X_test, y_test)  # Replace with actual test data

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 449us/step - accuracy: 0.8685 - loss: 0.3216


[0.3525087535381317, 0.8489583134651184]

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [29]:
# Make predictions on random rows from the test set
random_indices = np.random.choice(X_test.shape[0], 5, replace=False)
sample_X = X_test[random_indices]
sample_y = y_test.iloc[random_indices]

predictions = loaded_model.predict(sample_X)
predicted_classes = np.argmax(predictions, axis=1)

# Compare predictions with actual labels
for i in range(len(sample_X)):
    print(f"Sample: {sample_X[i]}, Predicted: {predicted_classes[i]}, Actual: {sample_y.iloc[i]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Sample: [-0.28682739 -1.41958965  1.00322257  1.08495342 -0.42007679 -1.41958965
  1.00322257 -1.41958965 -0.28682739], Predicted: 0, Actual: 0
Sample: [ 1.00322257  1.08495342  1.00322257 -1.41958965 -0.42007679 -1.41958965
 -0.28682739 -0.16731812  1.00322257], Predicted: 1, Actual: 1
Sample: [-0.28682739  1.08495342  1.00322257  1.08495342 -0.42007679 -0.16731812
 -0.28682739  1.08495342  1.00322257], Predicted: 0, Actual: 0
Sample: [ 1.00322257 -1.41958965 -0.28682739  1.08495342 -0.42007679  1.08495342
 -0.28682739 -0.16731812  1.00322257], Predicted: 0, Actual: 0
Sample: [-0.28682739  1.08495342 -0.28682739  1.08495342  0.93037142  1.08495342
 -0.28682739 -1.41958965 -1.57687736], Predicted: 1, Actual: 1


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [52]:
# Try adding more layers and neurons
model_improved = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(2, activation='softmax')  # Output layer
])

In [53]:
# Adjust learning rate
optimizer = Adam(learning_rate=0.0005)

# Compile the improved model
model_improved.compile(optimizer=optimizer,
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

# Train the model for more epochs
history_improved = model_improved.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, batch_size=32)

Epoch 1/150
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3520 - loss: 0.8012 - val_accuracy: 0.4427 - val_loss: 0.7093
Epoch 2/150
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 650us/step - accuracy: 0.5522 - loss: 0.6835 - val_accuracy: 0.6302 - val_loss: 0.6578
Epoch 3/150
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 624us/step - accuracy: 0.6610 - loss: 0.6399 - val_accuracy: 0.6510 - val_loss: 0.6346
Epoch 4/150
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 630us/step - accuracy: 0.6813 - loss: 0.6088 - val_accuracy: 0.6719 - val_loss: 0.6163
Epoch 5/150
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 626us/step - accuracy: 0.6745 - loss: 0.6100 - val_accuracy: 0.6979 - val_loss: 0.6012
Epoch 6/150
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 660us/step - accuracy: 0.7065 - loss: 0.5897 - val_accuracy: 0.7083 - val_loss: 0.5881
Epoch 7/150
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - accuracy: 0.7145 - loss: 0.5588 - val_accuracy: 0.7292 - val_loss: 0.5732
Epoch 8/150
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 634us/step - accuracy: 0.7301 - loss: 0.5517 - val_accura

In [54]:
# Evaluate the improved model
test_loss_improved, test_accuracy_improved = model_improved.evaluate(X_test, y_test)
print(f"Improved Test Loss: {test_loss_improved}")
print(f"Improved Test Accuracy: {test_accuracy_improved}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 710us/step - accuracy: 0.9481 - loss: 0.1008
Improved Test Loss: 0.10410038381814957
Improved Test Accuracy: 0.953125


**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# Adding layers improved the model's performance.
# Adjusting the learning rate to 0.0005 provided a stable convergence.
# Training for 100 epochs reached the desired performance threshold.